# LA-ConGA Physics 2022

# Módulo de Datos

Estudiante: Oscar, Altuve Pabón

Mattermost: @altuos
### Tarea 2. Ejercicio 1.

Utilizando el arreglo de puntos de la lámina 39:
1.	Construir una función $\chi^2(p_0,p_1)$ correspondiente a un modelo lineal $y=p_0+p_1x$;
2.	Asignar valores numéricos iniciales “razonables” a $p_0$ y $p_1$;
3.	Determinar (p.e. con una doble bucle) los valores $\hat{p_0}$ y $\hat{p_1}$ que minimizan la función $\chi^2$;
4.	Fijando el valor $\hat{p_0}$, graficar la variación de $\chi^2$ en función de $p_1$ alrededor de su valor mínimo;
5.	Fijando el valor de $\hat{p_1}$, graficar la variación de $\chi^2$ en función de $p_0$ alrededor de un valor mínimo;
6.	En ambos casos, determinar los intervalos correspondientes a $\Delta\chi^2<1$;
7.	Comparar con el resultado indicado en la lámina siguiente;
8.	Evaluar la p-value, e interpretar su valor;
9.	Comparar con el resultado producido por tu herramienta favorita, e interpretar las posibles diferencias.

In [ ]:
#include<TFile.h>
#include<TH1.h>
#include<TMath.h>
#include<math.h>
#include<TF1.h>
#include<TLine.h>
#include<TPoint.h>
#include<iostream>
#include<string>
#include<stdio.h>